In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of methods.runner failed: Traceback (most recent call last):
  File "/Users/enrico/Documents/Faculdade/TCC Local/Projeto/.venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/Users/enrico/Documents/Faculdade/TCC Local/Projeto/.venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 455, in superreload
    module = reload(module)
  File "/usr/local/Cellar/python@3.9/3.9.5/Frameworks/Python.framework/Versions/3.9/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 855, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/Users/enrico/Documents/Faculdade/TCC Local/Projeto/methods/runner.py", line 7, in <module>
    from .lstm import lstm, stacked_lstm
ImportError: cannot im

In [2]:
%pip install statsmodels --upgrade

You should consider upgrading via the '/Users/enrico/Documents/Faculdade/TCC Local/Projeto/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [17]:
from methods.ar import ar, arima
from methods.naive import average_forecast, all_true, all_false
from methods.lstm import lstm
from methods.mlp import mlp
from methods.mlp import mlp_keras
from methods.cnn import cnn
from methods.classifiers import knn, gaussian, random_forest
from methods.runner import Runner
import functools


In [7]:
# ar -> n_steps = 2, 4, 8
# arima -> n_steps = 2, 4, 8; order = 2, 4, 8; differencing = 1, 2, 3
# KNN -> n_steps = 2, 4, 8; n_neighbours = 2, 4, 8; distances = 1, 2, 3, 4, 5; 
# MLP -> n_steps = 2, 4, 8; epochs = 50, 100, 200; layers = 1, 2, 3; cells = 50, 100, 200
# LSTM -> n_steps = 2, 4, 8; epochs = 50, 100, 200; layers = 1, 2, 3; cells = 50, 100, 200

models = {}
epochs = [50, 100, 200]
layers = [1, 2, 3]
cells = [50, 100, 200]
orders = [2, 4, 8]
differencing = [1, 2, 3]
n_neighbours = [2, 4, 8]
distances = [1, 2, 3, 4, 5]

for order in orders:
	for diff in differencing:
		models[f"ARIMA_{order}_{diff}"] = functools.partial(arima, order=order, differencing=diff)

for n in n_neighbours:
	for d in distances:
		models[f"KNN_{n}_{d}"] = functools.partial(knn, n_neighbors=n, distance=d)

for epoch in epochs:
	for layer in layers:
		for cell in cells:
			models[f"MLP_{epoch}_{layer}_{cell}"] = functools.partial(mlp_keras, epochs=epoch, layers=layer, cells=cell)

for epoch in epochs:
	for layer in layers:
		for cell in cells:
			models[f"LSTM_{epoch}_{layer}_{cell}"] = functools.partial(lstm, epochs=epoch, layers=layer, cells=cell)

In [5]:
# models = {
# 	"Naive": average_forecast,
# 	"All True": all_true,
# 	"All False": all_false,
# 	# "AR": ar,
# 	"ARIMA": arima,
# 	# "MLP": mlp,
# 	# "KNN": knn,
# 	# "Gaussian": gaussian,
# 	# "RandomForest": random_forest,
# 	# "MLP Keras": mlp_keras,
# 	# "CNN": cnn,
# 	# "LSTM": lstm,
# 	# "Stacked LSTM": stacked_lstm
# }

In [12]:
models = {}
# models["MLP_200_3_200"] = functools.partial(mlp_keras, epochs=200, layers=3, cells=200)
models["LSTM_100_1_100"] = functools.partial(lstm, epochs=10, layers=1, cells=100, verbose=1)
# models["All True"] = all_true
# models["All False"] = all_false

In [25]:
def callback(self):
	print(self)

runner = Runner(models, callback = callback)
# runner.run_all("accuracy")
# runner.run("accuracy")


runner.run_transfer("accuracy")
# runner.continue_run("202204211108", "accuracy")

Trainning Brazil.csv (30/253) with shape (514, 34)
Epoch 1/10
8/8 [==============================] - 5s 12ms/step - loss: 0.6749 - accuracy: 0.6708
Epoch 2/10
8/8 [==============================] - 0s 18ms/step - loss: 0.6597 - accuracy: 0.6792
Epoch 3/10
8/8 [==============================] - 0s 13ms/step - loss: 0.6468 - accuracy: 0.6667
Epoch 4/10
8/8 [==============================] - 0s 11ms/step - loss: 0.6343 - accuracy: 0.6708
Epoch 5/10
8/8 [==============================] - 0s 13ms/step - loss: 0.6209 - accuracy: 0.6833
Epoch 6/10
8/8 [==============================] - 0s 13ms/step - loss: 0.6056 - accuracy: 0.7125
Epoch 7/10
8/8 [==============================] - 0s 14ms/step - loss: 0.5887 - accuracy: 0.7375
Epoch 8/10
8/8 [==============================] - 0s 13ms/step - loss: 0.5684 - accuracy: 0.7625
Epoch 9/10
8/8 [==============================] - 0s 11ms/step - loss: 0.5453 - accuracy: 0.7875
Epoch 10/10
8/8 [==============================] - 0s 15ms/step - loss: 0.52